In [1]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2___5-0___5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "介绍一下上海交通大学"
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_data = model.generate(
    **model_inputs,
    max_new_tokens=512,
    output_logits=True,
    return_dict_in_generate=True
)

generated_ids = generated_data['sequences']

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

'上海交通大学（Shanghai Jiaotong University），简称“上交大”，是中华人民共和国的综合性研究型大学，位于中国上海市。它是中国最早设立的高等学府之一，也是全球著名的多学科综合大学之一。\n\n上海交通大学拥有超过140年的历史，前身包括上海技术学院、上海师范学院和上海机器学院等。学校在教育、科研、社会服务等方面都有显著成就，特别是在计算机科学与技术、材料科学、医学等领域具有较高的国际影响力。\n\n上海交通大学的特色专业包括但不限于：\n\n1. 计算机科学与技术（CST）：专注于计算机硬件、软件及其相关领域。\n2. 材料科学与工程（MSE）：涵盖金属、塑料、陶瓷等多个材料领域的研究。\n3. 医学（MD/MBA）：培养医学生并提供管理学位课程。\n4. 环境科学与工程（ESE）：侧重环境科学、生态学及环境工程。\n5. 航空宇航科学与技术（AIST）：面向航空航天工程的研究和发展。\n\n上海交通大学致力于为学生提供卓越的学习体验，并通过各种教学方法支持学生的学术发展。学校的国际化程度高，吸引了来自世界各地的优秀学生，为他们提供了丰富的学习机会。\n\n上海交通大学作为中国的顶尖高等教育机构之一，在国内外享有盛誉，其毕业生深受雇主欢迎，被誉为“知识的摇篮”。随着科技的发展和社会的进步，上海交通大学将继续加强科技创新和人才培养，以应对未来挑战，推动科技进步和社会进步。'

In [32]:
generated_data.keys()

odict_keys(['sequences', 'logits', 'past_key_values'])

In [33]:
len(generated_data['logits'])

314

In [34]:
len(generated_data['sequences'][0])

346

In [37]:
import torch
from torch.nn.functional import softmax

decode_text = ''
# 假设logits_list是您提供的张量列表
logits_list = generated_data['logits']

# 将所有张量转移到CPU，并应用softmax
probabilities_list = [softmax(logits.cpu(), dim=1) for logits in logits_list]

# 获取每个样本的最大概率及其索引（即预测类别）
predictions = [torch.max(probs, dim=1) for probs in probabilities_list]

# 打印预测类别和对应的最大概率
for i, (values, indices) in enumerate(predictions):
    decode_text = decode_text + str(tokenizer.decode([indices.item()], skip_special_tokens=True))
    print(f"Sample {i+1}: Predicted class index = {indices.item()}, Decode word = {tokenizer.decode([indices.item()], skip_special_tokens=True)}, Probability = {values.item():.4f}")

print("\n=============Decode text=================")
print(decode_text)

Sample 1: Predicted class index = 100633, Decode word = 上海, Probability = 0.8816
Sample 2: Predicted class index = 115246, Decode word = 交通大学, Probability = 0.9890
Sample 3: Predicted class index = 9909, Decode word = （, Probability = 0.3225
Sample 4: Predicted class index = 2016, Decode word = Sh, Probability = 0.6653
Sample 5: Predicted class index = 30070, Decode word = anghai, Probability = 0.9694
Sample 6: Predicted class index = 619, Decode word =  J, Probability = 0.9731
Sample 7: Predicted class index = 22516, Decode word = iao, Probability = 0.7039
Sample 8: Predicted class index = 354, Decode word = ot, Probability = 0.9702
Sample 9: Predicted class index = 644, Decode word = ong, Probability = 0.9985
Sample 10: Predicted class index = 3822, Decode word =  University, Probability = 0.8916
Sample 11: Predicted class index = 11, Decode word = ,, Probability = 0.3230
Sample 12: Predicted class index = 102500, Decode word = 简称, Probability = 0.6295
Sample 13: Predicted class inde